# fix micropal 4 files

In [115]:
import sys
sys.path.append('../../../')
from pathlib import Path

import pandas as pd
import shutil
import numpy as np

from config import CLEAN_DATA_DIR, OUTPUT_DIR
from scripts.normalize_data import (
    normalize_sample_col, 
    normalize_expedition_section_cols, 
    csv_cleanup,
    update_metadata,
    fetch_unique_column_names,
    append_set,
    filter_existing_set,
    normalize_columns,
    add_missing_columns,
)

In [116]:
base_dir = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 



In [117]:
def log_df(df, row_count=5):
    print(df.shape)
    return df.head(row_count)


## check basic columns

check if files have basic columns

In [118]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [119]:
bad_files = set()
for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
        
    
    path = base_dir/file
    df = pd.read_csv(path, dtype=str, nrows=0)
    cols = df.columns
    if ('Sample' in cols or 'Label ID' in cols) or (
        'Exp' in cols
        and 'Hole' in cols 
        and 'Site' in cols 
        and 'Core' in cols 
        and 'Type' in cols 
        and 'Section' in cols
        ):
        pass
    else:
        bad_files.add(path)
        

        
len(bad_files)

124

In [120]:
# bad_files

# prep work

## get all files with intervals

create directory for files with intervals

In [121]:
columns_all = set()

for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
        
    fetch_unique_column_names(base_dir/file, columns_all)

In [122]:
interval_all=set()

append_set(interval_all, r".*?\binterval\b.*?", columns_all)

In [123]:
interval_all

{'Bottom interval (cm)',
 'Core, Section, Interval',
 'Core, section, interval',
 'Core, section, interval (cm)',
 'Expedition, site, hole, core, section, interval (cm):',
 'Interval (bottom)',
 'Interval (top)',
 'Interval Bot (cm) on SHLF',
 'Interval Top (cm) on SHLF',
 'Top interval (cm)',
 'bottom interval (cm)',
 'interval (cm)',
 'top interval (cm)'}

copy interval files

In [124]:
intervals = {
 'Core, Section, Interval',
 'Core, section, interval',
 'Core, section, interval (cm)',
 'Expedition, site, hole, core, section, interval (cm):',
 'interval (cm)'
}

for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
    path = Path(base_dir/file)

        
    df = pd.read_csv(path, dtype=str)
    for col in df.columns:
        if col in intervals:   
            new_path = Path('..', '..', '..', 'output', 'tmp', 'intervals_lims_4', path.name)
            # shutil.copy(path, new_path)


## create file for header depths

In [125]:
columns_all = set()

for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
        
    fetch_unique_column_names(base_dir/file, columns_all)

In [126]:
top_all = set()
bottom_all = set()
depth = set()
interval_all=set()

append_set(depth, r".*?\bdepth\b.*?", columns_all)
append_set(top_all, r".*?\btop\b.*?", columns_all)
append_set(bottom_all, r".*?\bbottom\b.*?", columns_all)
append_set(interval_all, r".*?\binterval\b.*?", columns_all)

In [127]:
cols = []
for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
        
    keep_cols = [file]
    path = Path(base_dir/file)
    df = pd.read_csv(path, dtype=str)
    for col in df.columns:
        if col in top_all or col in bottom_all or col in depth or col in interval_all:
            keep_cols.append(col)
    cols.append(keep_cols)
            
    

In [128]:
df = pd.DataFrame(cols, columns=['file','top','bottom','top depth','bottom depth','depth','interval',7,8,9])
log_df(df)

(137, 10)


,file,top,bottom,top depth,bottom depth,depth,interval,7,8,9
0,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,"Core, section, interval (cm)",Depth (csf),None,None,None,None,None,None,None
1,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,Top (cm),Bottom (cm),None,None,None,None,None,None,None
2,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,"Expedition, site, hole, core, section, interva...",Bottom depth CSF-B (m):,Top depth CSF-B (m):,None,None,None,None,None,None
3,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,Depth (cm),Top Depth (CSF m),None,None,None,None,None,None,None
4,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,Depth top CSF-A (m),Depth bottom CSF-A (m),None,None,None,None,None,None,None


In [50]:
# df.to_csv('../../../output/normalized_data/lims_4_depth_headers.csv', index=False)

manually fix lims_4_depth_headers

# update files

## fix bad header row

In [152]:
file = '../../../output/cleaned_data/LIMS/Micropal_CSV_4/317_U1351_planktic_forams.csv'
df = pd.read_csv(file, header=1, dtype=str)

log_df(df)

(232, 49)


,Expedition,Hole,Core,Core Type,Section,Top Depth [CSF m],Foraminferal preservation,Foraminiferal abundance,Planktic foraminiferal %,Globigerina spp.,...,Zeaglobierina woodi,Echinoid plate fragments,Echinoid spines,Fish teeth,Ostracods,Shell fragments,Micro-mollusks,Baranacle fragments,Bryozoan fragments,Remarks
0,317,U1351A,1,H,CC,2.56,G,A,13,X,...,NaN,NaN,F,NaN,F,A,NaN,NaN,NaN,NaN
1,317,U1351A,2,H,CC,11.44,G,A,17,X,...,NaN,NaN,NaN,NaN,NaN,A,A,NaN,NaN,NaN
2,317,U1351A,3,H,CC,18.15,G,A,26,X,...,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN
3,317,U1351A,4,H,CC,25.08,G,A,29,X,...,NaN,NaN,F,NaN,NaN,A,NaN,NaN,NaN,NaN
4,317,U1351A,5,H,CC,27.50,G,A,8,X,...,NaN,NaN,F,NaN,F,A,NaN,NaN,NaN,NaN


In [153]:
df.to_csv(file, index=False)

## rename exp..type columns

In [154]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [155]:
temp_files = [
    '321-U1337C_109_T08_rads.csv', 
    '321_U1337B_109_T07_rads.csv', 
    '321-U1337D_109_T09_rads.csv',
    '320_U1337A_109_T06_rads.csv'
]

for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
 
    path = Path(base_dir/file)
    df = pd.read_csv(path, dtype=str)
    
 
    if  path.name in ['317_U1351_benthic_forams.csv', '317_U1351_diatoms.csv']:
        if 'Hole.1' in df.columns:
            del df['Hole']
            del df['Exp']
            df.rename(columns={'Hole.1': 'Hole', 'Exp.1' : 'Exp'}, inplace=True)
            
    df.rename(columns={'Expedition' : 'Exp', 
                       'section': 'Section', 
                       'Core Type': 'Type',
                       'Core type': 'Type'}, inplace=True)

    if path.name in temp_files:
        df.rename(columns={'Core, section': 'Sample'}, inplace=True)
   
    
    df = csv_cleanup(df, path)
    df.to_csv(path, index=False)
    

## add missing expedition

In [156]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [157]:
for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
 
    path = Path(base_dir/file)
    df = pd.read_csv(path, dtype=str)
    
    if path.name == '323_U1339_palynology.csv':
        df['Exp'] = '323'
    if path.name == '323_U1339_dinoflagellates.csv':
        df['Exp'] = '323'
    if path.name == '321_U1338_nannofossils.csv':
        df['Sample'] = '321-' + df['Sample']

    
    df = csv_cleanup(df, path)
    df.to_csv(path, index=False)

## split columns

In [158]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [159]:
def split_column(df, column, new_columns, delim='-'):
    df[new_columns] = df[column].str.split(delim, expand=True)


def split_core(df):    
    df['Core'] = df['Core temp'].str.extract(r'([0-9]+)', expand=True)
    df['Type'] = df['Core temp'].str.extract(r'([a-zA-Z]+)', expand=True )

    
def split_core_interval(df, col):
    split_column(df, col, ['Core, section', 'temp_interval'], ', ')
    split_column(df, 'Core, section', ['Core temp', 'Section'])
    split_core(df)
    split_interval(df)

    del df['Core, section'] 
    del df['Core temp'] 
    
def split_sample_interval(df, col):
    split_column(df, col, ['Sample', 'temp_interval'], ', ')
    split_interval(df)
      
    
def split_interval(df, delim='-'):
    df['temp_interval'] = df['temp_interval'].str.replace(' cm', '')
    df['temp_interval'] = df['temp_interval'].str.replace('cm', '')
    df.loc[df['temp_interval'].isna(), 'temp_interval'] = delim

    split_column(df, 'temp_interval', ['Top [cm]', 'Bottom [cm]'], delim)
    
    del df['temp_interval'] 
    

def log_files(cols, file):
    if 'Exp.1' in cols:
        print('Exp.1', file)    
    if 'Site.1' in cols:
        print('Site.1', file)   
    if 'Hole.1' in cols:
        print('Hole.1', file)
    if 'Core.1' in cols:
        print('Core.1', file) 
    if 'Section.1' in cols:
        print('Section.1', file)
        
        
    if ('Exp' in cols) and ('Hole' in cols) and ('Site' in cols) and ('Core' in cols) and ('Type' in cols) and ('Section' in cols):
        pass
    elif 'Sample' in cols:
        pass
    elif 'Core, Section, Interval' in cols:
        # TODO: have PIs fix column
        print('Core, Section, Interval', path)
    elif 'Core, section, interval' in cols:
        # TODO: have PIs fix column
        print('Core, section, interval', path)
    else:
        pass
        # print('invalid:', file)

### split intervals

In [160]:
for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
 
    path = Path(base_dir/file)
    df = pd.read_csv(path, dtype=str)
    
    
    # split interval into separate columns 
    if 'Core, section, interval (cm)' in df.columns:
        interval_col = 'Core, section, interval (cm)' 
        if path.name == '320_U1331_benthic_forams.csv':
            df.loc[df[interval_col] == "10H7, 44-46 cm", interval_col] = "10H-7, 44-46 cm"
            df.loc[df[interval_col] == "10H7, 47-49 cm", interval_col] = "10H-7, 47-49 cm"
            
            split_core_interval(df, interval_col) 

        elif path.name == '320_U1336B_108_T10_planktic forams.csv':
            df[interval_col] = df[interval_col].str.replace('–', '-')
            split_sample_interval(df, interval_col)
        else:
            split_core_interval(df, interval_col) 
            
    elif 'Core, Section, Interval'  in df.columns:
        interval_col = 'Core, Section, Interval' 
        if path.name == '323_U1339A_diatoms.csv':
            df.loc[df[interval_col] == "4H-1-150-152", interval_col] = "4H-1, 150-152"
            df.loc[df[interval_col] == "4H-3-150-156", interval_col] = "4H-3, 150-156"
            df.loc[df[interval_col] == "4H-3-156-159", interval_col] = "4H-3, 156-159"
            
        elif path.name == '323_U1339B_diatoms.csv':
            df.loc[df[interval_col] == "9H-2-45cm", interval_col] = "9H-2, 45"
            df.loc[df[interval_col] == "9H-4-45cm", interval_col] = "9H-4, 45"

        elif path.name == '323_U1339C_diatoms.csv':
            df.loc[df[interval_col] == "10H-1-25cm", interval_col] = "10H-1, 25cm"
            df.loc[df[interval_col] == "10H-2-25cm", interval_col] = "10H-2, 25cm"
            
        split_core_interval(df, interval_col)
        
    elif 'Core, section, interval'  in df.columns:
        interval_col = 'Core, section, interval' 
        split_core_interval(df, interval_col)
    
    elif 'Expedition, site, hole, core, section, interval (cm):' in df.columns:
        interval_col = 'Expedition, site, hole, core, section, interval (cm):'
        split_sample_interval(df, interval_col)
    
        
    elif 'interval (cm)' in df.columns:
        interval_col = 'interval (cm)'
        df[interval_col] = df[interval_col].str.replace('–', '-')
        df[interval_col] = df[interval_col].str.replace('-', '-')
        df['temp_interval'] = df[interval_col]

        split_interval(df)
        
    elif path.name == '321_U1338_nannofossils.csv':
        interval_col = 'Sample'
        df.loc[df[interval_col] == "321-U1338A-2H-2,24", interval_col] = "321-U1338A-2H-2, 24"
        df.loc[df[interval_col] == "321-U1338A-5H-5,70", interval_col] = "321-U1338A-5H-5, 70"
        df.loc[df[interval_col] == "321-U1338A-24H-4-80", interval_col] = "321-U1338A-24H-4, 80"

        split_sample_interval(df, interval_col)

    cols = list(df.columns)
    log_files(cols, file)

            
    df = csv_cleanup(df, path)
    df.to_csv(path, index=False)
    


### split core

In [161]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [162]:
for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
 
    path = Path(base_dir/file)
    df = pd.read_csv(path, dtype=str)
    

    # edit column so that we can split it into separate columns  
    if path.name == '323_U1339_radiolarians.csv':
        df['Hole, Core, Section temp'] = df['Hole, Core, Section'].str.replace(' ', '-') 
        
    elif path.name == '323_U1343_silicoflagellates.csv':
        df.rename(columns={'Unnamed: 3': 'Core, Type, Section'}, inplace=True)
        df['Core, Type, Section temp'] = df['Core, Type, Section'].str.replace('^(\d+)([A-Z-]+)$', r'\1-\2', regex=True)
    
    
    # split core into separate columns 
    temp_files = [
        '323_U1339C_nannofossils.csv',
        '323_U1339C_benthic_forams.csv',
        '323_U1339B_nannofossils.csv',
        '320_U1335_planktic_forams.csv',
        '323_U1341B_benthic_forams.csv',
        '320_U1333_planktic_forams.csv',
        '323_U1341A_benthic_forams.csv',
        '323_U1339A_nannofossils.csv',
        '323_U1339_silicoflagellates.csv',
        '323_U1339A_benthic_forams.csv',
        '323_U1339B_benthic_forams.csv',
        '323_U1341A_ostracods.csv',
        '323_U1341C_benthic_forams.csv',
        '320_U1336A_planktic_forams.csv',
        '323_U1339D_benthic_forams.csv',
        '323_U1339D_nannofossils.csv'
    ]
    for temp_file in temp_files:
        if path.name == temp_file:
            df['Core temp'] = df['Core']
            split_core(df)
            del df['Core temp']
            
    
    # split column into multiple columns    
    target_columns = ['core, section', 'Core, section', 'Core, Section', 'Core,    section']
    for target_column in target_columns:
        if target_column in df.columns:
            split_column(df, target_column, ['Core','Section'])
            df['Core temp'] = df['Core']
            split_core(df) 
            del df['Core temp']


    if 'Hole, Core, Section temp' in df.columns:
        split_column(df, 'Hole, Core, Section temp', ['Hole', 'Core', 'Section'])
        df['Core temp'] = df['Core']
        split_core(df)
        del df['Hole, Core, Section temp']
        del df['Core temp']

    elif 'Core Type - Section' in df.columns:
        split_column(df, 'Core Type - Section', ['Type', 'Section'])
        
    elif 'Core, Type, Section temp' in df.columns:
        split_column(df, 'Core, Type, Section temp', ['Core', 'Type', 'Section'])
        del df['Core, Type, Section temp']
    
        
    cols = list(df.columns)
    
    # log_files(cols, file)
        
    df = csv_cleanup(df, path)
    df.to_csv(path, index=False)
    


### split site hole

In [163]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [164]:
def split_site_hole(df, col):
    df[['Site', 'Hole']] = df[col].str.extract(r'(U[0-9]+)([a-zA-Z])', expand=True)
    

for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
 
    path = Path(base_dir/file)
    df = pd.read_csv(path, dtype=str)

    if path.name == '323_U1344_Radiolarians.csv':
        df['Hole'] = 'U' + df['Hole']
        split_site_hole(df, 'Hole')
        
    elif path.name == '317-U1352_benthic_forams.csv':
        split_site_hole(df, 'Site')
    

    temp_files = [
        '321_U1337A_nannofossils.csv',
        '320_U1334_benthic_forams.csv',
        '317_U1352_planktic_forams.csv',
        '317_U1353_planktic_forams.csv',
        '317_U1351_nannofossils.csv',
        '317_U1351_planktic_forams.csv',
        '323_U1339_radiolarians.csv',
        '323_U1343_radiolarians.csv',
        '317_U1354_planktic_forams.csv',
        '317_U1354_benthic_forams.csv',
        '320_U1336A_benthic_forams.csv',
        '320_U1331_benthic_forams.csv',
        '317_U1353_diatoms.csv',
        '323_U1340A_dinoflagellates.csv',
        '320_U1332_benthic_forams.csv',
        '317_U1354_nannofossils.csv',
        '317_U1353_nannofossils.csv',
        '320_U1333_benthic_forams.csv',
        '317_U1354_diatoms.csv',
        '317_U1353_benthic_forams.csv',
        '323_U1340A_palynology.csv',
        '317_U1352_nannofossils.csv',
        '320_U1335_benthic_forams.csv',
    ]
    if path.name in temp_files:
        split_site_hole(df, 'Hole')
  
    
    df = csv_cleanup(df, path)
    df.to_csv(path, index=False)

## normalize depth headers

In [165]:
header_df = pd.read_csv('../../../output/normalized_data/lims_4_depth_headers.csv')
log_df(header_df)

(137, 12)


,file,top,bottom,top depth,bottom depth,depth,interval,7,8,9,Unnamed: 10,Unnamed: 11
0,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,Top [cm],Bottom [cm],Depth (csf),NaN,NaN,"Core, section, interval (cm)",NaN,NaN,NaN,NaN,NaN
1,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,Top (cm),Bottom (cm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,Top [cm],Bottom [cm],Top depth CSF-B (m):,Bottom depth CSF-B (m):,NaN,"Expedition, site, hole, core, section, interva...",NaN,NaN,NaN,NaN,NaN
3,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,NaN,NaN,Top Depth (CSF m),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,NaN,NaN,Depth top CSF-A (m),Depth bottom CSF-A (m),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
for index, row in header_df.iterrows():
    path = base_dir/row['file']
    df = pd.read_csv(path, dtype=str)
    df.rename(columns={row['top']: 'Top [cm]', 
                       row['bottom']: 'Bottom [cm]', 
                       row['top depth']: 'Top Depth [m]',
                       row['bottom depth']: 'Bottom Depth [m]'
    }, inplace=True)
    
    df = csv_cleanup(df, path)
    df.to_csv(path, index=False)

## add missing columns 

In [167]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [168]:
normalized_columns = [
    'Top [cm]',
    'Bottom [cm]',
    'Top Depth [m]',
    'Bottom Depth [m]',
]

exp_columns = [
    'Site',
    'Hole',
    'Core',
    'Type',
    'Section',
    'A/W'
]

for file in metadata['path']:
    if 'Micropal_CSV_4' not in file:
        continue
 
    path = Path(base_dir/file)
    df = pd.read_csv(path, dtype=str)
    
    for col in normalized_columns:
        if col not in df.columns:
            df[col] = np.nan
            
    if 'Exp' in df.columns:
        for col in exp_columns:
            if col not in df.columns:
                df[col] = np.nan
    
    
    df = csv_cleanup(df, path)
    df.to_csv(path, index=False)